In [32]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as sco

plt.style.use('fivethirtyeight')
np.random.seed(0)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [33]:
table = pd.read_csv('./data/data.csv')
table.head()

,SYMBOL,ASIANPAINT,HDFC,ITC
0,Date,NaN,NaN,NaN
1,2000-01-03,381.65,293.50,708.50
2,2000-01-04,385.55,304.05,712.35
3,2000-01-05,383.00,292.80,726.20
4,2000-01-06,377.50,296.45,784.30


In [34]:
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

In [35]:
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(3)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

In [36]:
returns = table.loc[:,table.columns!="SYMBOL"].pct_change()
date = table.loc[:,table.columns=="SYMBOL"]
returns = date.join(returns)
returns.head()


,SYMBOL,ASIANPAINT,HDFC,ITC
0,Date,NaN,NaN,NaN
1,2000-01-03,NaN,NaN,NaN
2,2000-01-04,0.010219,0.035945,0.005434
3,2000-01-05,-0.006614,-0.037000,0.019443
4,2000-01-06,-0.014360,0.012466,0.080006


In [37]:
mean_returns = returns.mean()
print(mean_returns)
cov_matrix = returns.cov()
print(cov_matrix)
num_portfolios = 25000
risk_free_rate = 0.0178

ASIANPAINT    0.000804
HDFC          0.000838
ITC           0.000322
dtype: float64
            ASIANPAINT      HDFC       ITC
ASIANPAINT    0.000499  0.000085  0.000062
HDFC          0.000085  0.000680  0.000124
ITC           0.000062  0.000124  0.000607


/var/folders/h2/_wt314sx1lv933t020qmfv400000gn/T/ipykernel_52695/2714011366.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_returns = returns.mean()


In [38]:
def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=table.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    
    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx],index=table.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)


In [39]:
display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)

ValueError: Shape of passed values is (3, 1), indices imply (4, 1)